In [1]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import time
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [2]:
tond = pd.read_csv("type_before_pre_ton.csv")

In [3]:
iot20 = pd.read_csv("iot2020.csv")

In [4]:
#5:5 비율을 맞추기 위해 ton-iot의 개수만큼 랜덤 추출
iot20 = iot20.sample(n=461043, replace=False, axis=0)

In [5]:
del tond['Unnamed: 0'] 

In [9]:
tond['type'].value_counts()

normal        300000
ddos           20000
backdoor       20000
scanning       20000
injection      20000
password       20000
xss            20000
ransomware     20000
dos            20000
mitm            1043
Name: type, dtype: int64

In [7]:
#feature명 통일
tond.rename(columns={'type':'Cat'})

,ts,src_port,dst_port,proto,service,duration,src_bytes,dst_bytes,conn_state,missed_bytes,...,http_request_body_len,http_response_body_len,http_status_code,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,Cat
0,1554198358,1883,52976.0,6,0,80549.530260,1762852.0,41933215.0,2,0.0,...,0.0,0.0,0,0,0,5,0,1.0,0,normal
1,1554198358,47260,15600.0,17,0,0.000000,0.0,0.0,1,0.0,...,0.0,0.0,0,0,0,0,0,0.0,0,normal
2,1554198359,1880,51782.0,6,0,0.000000,0.0,0.0,2,0.0,...,0.0,0.0,0,0,0,5,0,1.0,0,normal
3,1554198359,34296,10502.0,6,0,0.000000,0.0,0.0,2,0.0,...,0.0,0.0,0,0,0,0,0,0.0,0,normal
4,1554198362,46608,53.0,17,1,0.000549,0.0,298.0,4,0.0,...,0.0,0.0,0,0,0,4,0,1.0,0,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461038,1556340862,33108,80.0,6,0,0.000000,0.0,0.0,1,0.0,...,0.0,0.0,0,0,0,0,0,0.0,1,xss
461039,1556423390,37242,443.0,6,0,0.000000,0.0,0.0,2,0.0,...,0.0,0.0,0,0,0,0,0,0.0,1,ransomware
461040,1556436603,4444,49178.0,6,0,290.371539,101568.0,2592.0,2,0.0,...,0.0,0.0,0,0,0,0,0,0.0,1,backdoor
461041,1556540442,60816,443.0,6,0,23.190902,32.0,31.0,0,0.0,...,0.0,0.0,0,0,0,0,0,0.0,1,mitm


In [8]:
#공격명 통일
iot20 = iot20.replace('Normal', 'normal' )
iot20 = iot20.replace('DoS', 'dos' )
iot20 = iot20.replace('MITM ARP Spoofing', 'mitm' )
iot20 = iot20.replace('Scan', 'scanning' )

In [11]:
iot20 = iot20.drop(columns=[ 'Label','Sub_Cat','Flow_ID', 'Src_IP', 'Dst_IP',  'Timestamp', 'Src_IP', 'Src_Port', 'Dst_IP', 'Dst_Port'])

In [12]:
tond = tond.drop(columns=[ 'ts', 'src_port', 'dst_port'])

In [13]:
#feature명 통일
tond= tond.rename(columns={'duration':'Flow_Duration','src_pkts':'Tot_Fwd_Pkts', 'dst_pkts':'Tot_Bwd_Pkts', 'proto':'Protocol', 'src_bytes':'Fwd_Pkt_Len_Mean' , 'dst_bytes':'Bwd_Pkt_Len_Mean', 'src_ip_bytes':'Fwd_Pkts/s', 'dst_ip_bytes':'Bwd_Pkts/s', 'type':'Cat', 'ts':'Timestamp', 'src_port':'Src_Port', 'dst_port':'Dst_Port', 'label':'Label'})

In [25]:
#데이터 합치기
frames = [iot20, tond]
df_merge = pd.concat(frames)

In [31]:
df_merge = df_merge.drop(columns=['Label'])

KeyError: "['Label'] not found in axis"

In [30]:
#결측값 존재 여부 확인
df_merge.isna().sum()

Protocol                0
Flow_Duration           0
Tot_Fwd_Pkts            0
Tot_Bwd_Pkts            0
TotLen_Fwd_Pkts         0
                       ..
http_orig_mime_types    0
http_resp_mime_types    0
weird_name              0
weird_addl              0
weird_notice            0
Length: 104, dtype: int64

In [29]:
#결측값 채우기
tond.fillna(-1,inplace=True)
iot20.fillna(-1,inplace=True)
df_merge.fillna(-1,inplace=True)

In [41]:
#각 데이터셋에 없는 feature 추가
tond['Init_Bwd_Win_Byts']=np.NaN
tond['Flow_Pkts/s']=np.NaN
tond['Flow_IAT_Max']=np.NaN
tond['Bwd_IAT_Tot']=np.NaN

iot20['conn_state']=np.NaN
iot20['service']=np.NaN

In [42]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier()

In [43]:
X = df_merge[['Protocol', 'Flow_Duration', 'Fwd_Pkt_Len_Mean', 'Bwd_Pkt_Len_Mean', 'Tot_Fwd_Pkts', 'Fwd_Pkts/s', 'Tot_Bwd_Pkts', 'Bwd_Pkts/s', 'conn_state', 'Init_Bwd_Win_Byts', 'Flow_Pkts/s', 'Bwd_IAT_Tot', 'service', 'Flow_IAT_Max']]
Y = df_merge['Cat']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=10, test_size=0.2)

In [44]:
Z = iot20[['Protocol', 'Flow_Duration', 'Fwd_Pkt_Len_Mean', 'Bwd_Pkt_Len_Mean', 'Tot_Fwd_Pkts', 'Fwd_Pkts/s', 'Tot_Bwd_Pkts', 'Bwd_Pkts/s', 'conn_state', 'Init_Bwd_Win_Byts', 'Flow_Pkts/s', 'Bwd_IAT_Tot', 'service', 'Flow_IAT_Max']]
W = iot20['Cat']
Z_train, Z_test, W_train, W_test = train_test_split(Z, W, random_state=10, test_size=0.4)

In [45]:
start = time.time()
print('program start...')
print()

RF.fit(X_train, Y_train)  # fit() : 훈련 수행
print()
print('prediction:')
y_pred = RF.predict(X_test)  # predict() : 모델 훈련 후 예측
print(y_pred)
print()

print('Score:')
score = RF.score(X_test,Y_test)  # score() : 평가 점수 계산
print(score)

end = time.time()  # 현재 시각 구하기
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')
print("Classifiction Report :")
print(classification_report(Y_test, y_pred))

program start...


prediction:
['xss' 'Mirai' 'scanning' ... 'normal' 'Mirai' 'normal']

Score:
0.9242481753408018
program end...

time cost: 
76.99175906181335 seconds
Classifiction Report :
              precision    recall  f1-score   support

       Mirai       0.94      0.90      0.92     61138
    backdoor       1.00      1.00      1.00      3989
        ddos       0.98      0.96      0.97      3984
         dos       1.00      0.99      0.99     12853
   injection       0.97      0.96      0.97      3997
        mitm       0.65      0.45      0.53      5477
      normal       0.99      0.98      0.99     66009
    password       0.98      0.98      0.98      4012
  ransomware       0.86      0.93      0.89      4007
    scanning       0.64      0.84      0.73     15018
         xss       0.94      0.91      0.92      3934

    accuracy                           0.92    184418
   macro avg       0.90      0.90      0.90    184418
weighted avg       0.93      0.92      0.93    184